## Low-Tubal-Rank Smoothing Tensor Completion Imputer (LSTC-Tubal)

This notebook shows how to implement a LSTC-Tubal imputer on some real-world large-scale data sets. To overcome the problem of missing values within multivariate time series data, this method takes into account both low-rank structure and time series regression. Meanwhile, to make the model scalable, we also integrate linear transform into the LATC model. For an in-depth discussion of LATC-Tubal-imputer, please see [1].

<div class="alert alert-block alert-info">
<font color="black">
<b>[1]</b> Xinyu Chen, Yixian Chen, Lijun Sun (2020). <b>Scalable low-rank tensor learning for spatiotemporal traffic data imputation</b>. arXiv: 2008.03194. <a href="https://arxiv.org/abs/2008.03194" title="PDF"><b>[PDF]</b></a> <a href="https://doi.org/10.5281/zenodo.3939792" title="data"><b>[data]</b></a> 
</font>
</div>


In [1]:
import os
import random
import sys
import math
import pickle
import pandas as pd
import time
from tqdm import tqdm
import matplotlib.pyplot as plt
from torch.utils.data import DataLoader, Dataset ,ConcatDataset
sys.path.append(r'D:\WorkPath\Models\SAGAN')
from MyDataSet import MultiMaskTimeSeriesDataset

In [2]:
import numpy as np

def ten2mat(tensor, mode):
    return np.reshape(np.moveaxis(tensor, mode, 0), (tensor.shape[mode], -1), order = 'F')

def mat2ten(mat, dim, mode):
    index = list()
    index.append(mode)
    for i in range(dim.shape[0]):
        if i != mode:
            index.append(i)
    return np.moveaxis(np.reshape(mat, list(dim[index]), order = 'F'), 0, mode)

def unitary_transform(tensor, Phi):
    return np.einsum('kt, ijk -> ijt', Phi, tensor)

def inv_unitary_transform(tensor, Phi):
    return np.einsum('kt, ijt -> ijk', Phi, tensor)

def tsvt_unitary(tensor, Phi, tau):
    dim = tensor.shape
    X = np.zeros(dim)
    tensor = unitary_transform(tensor, Phi)
    for t in range(dim[2]):
        u, s, v = np.linalg.svd(tensor[:, :, t], full_matrices = False)
        r = len(np.where(s > tau)[0])
        if r >= 1:
            s = s[: r]
            s[: r] = s[: r] - tau
            X[:, :, t] = u[:, : r] @ np.diag(s) @ v[: r, :]
    return inv_unitary_transform(X, Phi)

from scipy.fftpack import dctn, idctn

def tsvt_dct(tensor, tau):
    dim = tensor.shape
    X = np.zeros(dim)
    tensor = dctn(tensor, axes = (2,), norm = 'ortho')
    for t in range(dim[2]):
        u, s, v = np.linalg.svd(tensor[:, :, t], full_matrices = False)
        r = len(np.where(s > tau)[0])
        if r >= 1:
            s = s[: r]
            s[: r] = s[: r] - tau
            X[:, :, t] = u[:, : r] @ np.diag(s) @ v[: r, :]
    return idctn(X, axes = (2,), norm = 'ortho')

def compute_mape(var, var_hat):
    return np.sum(np.abs(var - var_hat) / var) / var.shape[0]

def compute_rmse(var, var_hat):
    return  np.sqrt(np.sum((var - var_hat) ** 2) / var.shape[0])

def compute_mse(var, var_hat):
    return np.sum((var - var_hat) ** 2) / var.shape[0]



In [3]:
def imputer(dense_tensor, sparse_tensor, rho0, lambda0, epsilon, maxiter, 
            sparse_Psi = True, transform = "unitary"):
    """Low-Tubal-Rank Smoothing Tensor Completion, LSTC-Tubal-imputer."""
    
    dim = np.array(sparse_tensor.shape)
    dt = int(np.prod(dim) / dim[0])
    sparse_mat = ten2mat(sparse_tensor, 0)
    pos_missing = np.where(sparse_mat == 0)
    pos_test = np.where((dense_tensor != 0) & (sparse_tensor == 0))
    var = dense_tensor[pos_test]
    
    T = np.zeros(dim)                         # \boldsymbol{\mathcal{T}}
    Z = sparse_mat.copy()                     # \boldsymbol{Z}
    Z[pos_missing] = np.mean(sparse_mat[sparse_mat != 0])
    it = 0
    last_mat = sparse_mat.copy()
    snorm = np.linalg.norm(sparse_mat, 'fro')
    del dense_tensor, sparse_tensor, sparse_mat
    rho = rho0
    Phis = []
    if transform == "unitary":
        temp1 = ten2mat(mat2ten(Z, dim, 0), 2)
        _, Phi = np.linalg.eig(temp1 @ temp1.T)
        Phis.append(Phi)
        del temp1
    if lambda0 > 0:
        if sparse_Psi == True:
            from scipy import sparse
            from scipy.sparse.linalg import inv as inv
            Psi1 = sparse.coo_matrix((np.ones(dt - 1), (np.arange(0, dt - 1), np.arange(0, dt - 1))), 
                                     shape = (dt - 1, dt)).tocsr()
            Psi2 = sparse.coo_matrix((np.ones(dt - 1), (np.arange(0, dt - 1), np.arange(0, dt - 1) + 1)), 
                                     shape = (dt - 1, dt)).tocsr()
            temp0 = Psi2 - Psi1
            temp0 = temp0.T @ temp0
            Imat = sparse.coo_matrix((np.ones(dt), (np.arange(0, dt), np.arange(0, dt))), shape = (dt, dt)).tocsr()
            const = rho * inv(temp0 + rho * Imat / lambda0).todense() / lambda0
        elif sparse_Psi == False:
            Psi1 = np.append(np.eye(dt - 1), np.zeros((dt - 1, 1)), axis = 1)
            Psi2 = np.append(np.zeros((dt - 1, 1)), np.eye(dt - 1), axis = 1)
            temp0 = Psi2 - Psi1
            temp0 = temp0.T @ temp0
            const = rho * np.linalg.inv(temp0 + rho * np.eye(dt) / lambda0) / lambda0
        del Psi1, Psi2, temp0
    while True:
        rho = min(rho * 1.05, 1e5)
        if transform == "unitary":
            X = tsvt_unitary(mat2ten(Z, dim, 0) - T / rho, Phi, 1 / rho)
        elif transform == "dct":
            X = tsvt_dct(mat2ten(Z, dim, 0) - T / rho, 1 / rho)
        mat_hat = ten2mat(X, 0)
        temp = ten2mat(X + T / rho, 0)
        if lambda0 > 0:
            Z[pos_missing] = (temp @ const)[pos_missing]
        elif lambda0 == 0:
            Z[pos_missing] = temp[pos_missing]
        T = T + rho * (X - mat2ten(Z, dim, 0))
        tol = np.linalg.norm((mat_hat - last_mat), 'fro') / snorm
        last_mat = mat_hat.copy()
        it += 1
        if it % 10 == 0:
            if transform == "unitary":
                temp1 = ten2mat(mat2ten(Z, dim, 0) - T / rho, 2)
                _, Phi = np.linalg.eig(temp1 @ temp1.T)
                Phis.append(Phi)
                del temp1
        if it % 5 == 0:
            print('Iter: {}'.format(it))
            print('Tolerance: {:.6}'.format(tol))
            print('MAPE: {:.6}'.format(compute_mape(var, X[pos_test])))
            print('RMSE: {:.6}'.format(compute_rmse(var, X[pos_test])))
            print()
        if (tol < epsilon) or (it >= maxiter):
            break

    print('Total iteration: {}'.format(it))
    print('Tolerance: {:.6}'.format(tol))
    print('Imputation MAPE: {:.6}'.format(compute_mape(var, X[pos_test])))
    print('Imputation RMSE: {:.6}'.format(compute_rmse(var, X[pos_test])))
    print()
    
    return X, Phis ,compute_mse(var, X[pos_test]) ,compute_mape(var, X[pos_test])



In [4]:
def LSTC_T(data : MultiMaskTimeSeriesDataset, iter ,history_len = 20):

    test = np.random.randint(history_len*data.num_masks, len(data)-1 , iter)
    total_MSE = []
    total_MAPE = []
    for i in range(iter):
        index = test[i]

        dense_tensor = data.get_historical_data(index, history_len)[0]
        masks = data.get_historical_data(index, history_len)[1][:,index % data.num_masks,:,:]  # (num_day,time,station)
        sparse_tensor = dense_tensor * masks
        
        dense_tensor = dense_tensor.transpose(2,1,0)  # station , time, num of day
        sparse_tensor = sparse_tensor.transpose(2,1,0)
        dim = dense_tensor.shape

        start = time.time()
        dim = np.array(sparse_tensor.shape)

        rho = 2e-3
        epsilon = 1e-4
        maxiter = 100
        
        c = 0.5
        lambda0 = c * rho

        tensor_hat, Phis , MSE, MAPE= imputer(dense_tensor, sparse_tensor, rho, lambda0, epsilon, maxiter)
        
        total_MAPE.append(MAPE)
        total_MSE.append(MSE)
    return np.mean(total_MSE),np.mean(total_MAPE)

In [ ]:
project_path = r'D:\WorkPath\Models\ImputeFormer'
test_path = os.path.join(project_path , r'Data\source_test_PEMS04') 
test_files = os.listdir(test_path)
test_files = [os.path.join(test_path, file) for file in test_files]

test_record = {'data_name':[],'MSE_test_loss':[] , 'MAPE_test_loss':[]}

for file_path in test_files:
    with open(file_path, 'rb') as f:
        test_data = pickle.load(f)
        # 是否标准化原始数据
        # test_data.data=(test_data.data-np.mean(test_data.data))/np.std(test_data.data)

    total_MSE,total_MAE=LSTC_T(test_data,10 , 7)
    test_record['data_name'].append(file_path)
    test_record['MSE_test_loss'].append(np.mean(total_MSE))
    test_record['MAPE_test_loss'].append(np.mean(total_MAE))
test_record = pd.DataFrame(test_record)
test_record['route']=test_record['data_name'].apply(lambda x :x.split('_')[5])
test_record['start']=test_record['data_name'].apply(lambda x :x.split('_')[-3])
test_record['miss_rate']=test_record['data_name'].apply(lambda x :x.split('_')[-2])
test_record['type']=test_record['data_name'].apply(lambda x :x.split('_')[-1][:-4])
test_record=test_record[['route','start','miss_rate','type','MSE_test_loss','MAPE_test_loss']]
test_record=test_record.sort_values(['route','type','start'])
test_record